<a href="https://colab.research.google.com/github/reparadise/MachinelearningProjectAifel/blob/main/%EB%B2%88%EC%97%AD%EA%B8%B0%EB%8B%A8%EC%96%B4%EC%88%98%EC%A4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
pip install --upgrade tensorflow==2.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.3 kB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949


In [1]:
import tensorflow
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

print(tensorflow.__version__)

2.9.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
file_path = '/content/drive/MyDrive/Colab Notebooks1/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t') #t tab
print('전체 샘플의 수:', len(lines))
lines.sample(5)

# 파일 열기
#with open(file_path, "r") as f:
#    raw_corpus = f.read().splitlines().rstrip() # 텍스트를 라인 단위로 끊어서 list 형태로 읽어온다.

#print(raw_corpus[:12])


전체 샘플의 수: 194513


,eng,fra,cc
97020,They're part-time employees.,Ce sont des employés à temps partiel.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
109396,I won't go to school tomorrow.,Je n'irai pas à l'école demain.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
54061,Are you afraid of that?,Avez-vous peur de ça ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
64574,I've been misunderstood.,Je fus mal compris.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
94649,I want to make it up to you.,Je veux vous faire la cour.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    #sentence = sentence.lower().strip() # 1
    #sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    #sentence = re.sub(r'[" "]+', " ", sentence) # 3
    # sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    # sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> This @_is ;;;sample        sentence. <end>


In [5]:
# 구두점 제거 # #library that contains punctuation
# import string
# string.punctuation
# #defining the function to remove punctuation
# def remove_punctuation(text):
#     punctuationfree="".join([i for i in text if i not in string.punctuation])
#     return punctuationfree
# #storing the puntuation free text
# data['clean_msg']= data['v2'].apply(lambda x:remove_punctuation(x))
# data.head()

1. 구두점(Punctuation)을 단어와 분리해주세요. why, how
2. 소문자로 바꿔주세요.
3. 띄어쓰기 단위로 토큰를 수행하세요.

In [6]:
lines = lines[['eng', 'fra']][:30000] # 3만개 샘플 사용
lines.sample(5)

,eng,fra
15430,She is a runner.,C'est une coureuse.
393,Show me.,Montrez-moi !
4821,I could walk.,Je pouvais marcher.
26550,Tom never gave up.,Tom ne renonça jamais.
11302,Let Tom answer.,Laisse Tom répondre.


In [7]:
# seq2seq 동작을 위해서 디코더의 입력과 예측에 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines[100:120]

전체 샘플의 수 : 30000


,eng,fra
100,Hop in.,\t Montez. \n
101,Hug me.,\t Serre-moi dans tes bras ! \n
102,Hug me.,\t Serrez-moi dans vos bras ! \n
103,I fell.,\t Je suis tombée. \n
104,I fell.,\t Je suis tombé. \n
105,I fled.,\t J'ai fui. \n
106,I knit.,\t Je tricote. \n
107,I know.,\t Je sais. \n
108,I left.,\t Je suis parti. \n
109,I left.,\t Je suis partie. \n


In [8]:
# from nltk.tokenize import sent_tokenize
# 출처: https://zamezzz.tistory.com/314 [배워가는블로거:티스토리]
# sent_tokenize

In [9]:
eng_tokenizer =tensorflow.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
)
# analyzer=None,
#     )
#     **kwargs
# )  

eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[21], [21], [21]]

In [10]:
lines.eng[100:9674]

100             Hop in.
101             Hug me.
102             Hug me.
103             I fell.
104             I fell.
             ...       
9669    Coffee, please.
9670    Come back here.
9671    Come back here.
9672    Come back home.
9673    Come back home.
Name: eng, Length: 9574, dtype: object

In [20]:
fra_tokenizer =Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-.:;<=>?@[\\]^_`{|}~t',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None, 
  )  #  단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 63, 2], [1, 379, 2], [1, 702, 2]]

In [12]:
class SomeClass:
    def __init__(self,something):
        self.something = something

    def some_function(self):
        print(self.something)
        
a = SomeClass("some_value")
a.some_function()
#출처: https://engineer-mole.tistory.com/190 [매일 꾸준히, 더 깊이:티스토리]#

some_value


In [13]:
lines.fra[5:10]

5                           \t Cours ! \n
6                          \t Courez ! \n
7    \t Prenez vos jambes à vos cous ! \n
8                            \t File ! \n
9                           \t Filez ! \n
Name: fra, dtype: object

In [14]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4493
프랑스어 단어장의 크기 : 9410


In [15]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 6
프랑스어 시퀀스의 최대 길이 13


In [16]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 30000
영어 단어장의 크기 : 4493
프랑스어 단어장의 크기 : 9410
영어 시퀀스의 최대 길이 6
프랑스어 시퀀스의 최대 길이 13


인코더의 입력으로 사용되는 영어 시퀀스와 달리, 프랑스어 시퀀스는 2가지 버전으로 나누어 준비해야 합니다. 하나는 디코더의 출력과 비교해야 할 정답 데이터로 사용해야 할 원래 목적에 따른 것입니다. 그리고 다른 하나는 이전 스텝에서 언급했던 교사 강요(Teacher forcing)을 위해 디코더의 입력으로 사용하기 위한 것입니다.

이때, 디코더의 입력으로 사용할 시퀀스는 < eos >토큰이 필요가 없고, 디코더의 출력과 비교할 시퀀스는 < sos >가 필요가 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야 합니다.

In [21]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 

# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]